In [ ]:
import os 
import re
import math
import numpy as np
import ast
from functools import cmp_to_key
from collections import defaultdict
from itertools import combinations

PATH = '/Users/xuyaozhang/Desktop/AdventofCode2022/input'

def def_value():
    return [0, []]

def splitstr(x):
    chunks, chunk_size = len(x), 2
    return [ x[i:i+chunk_size] for i in range(0, chunks, chunk_size) ]

def read_txt(filename):
    res = defaultdict(def_value)
    
    with open(filename) as data_file:
            
        lines = data_file.read().split('\n') 
        for line in lines:
            nums = re.findall('\d+', line)
            valves = re.sub('[^A-Z]', '', line)
            res[valves[1:3]] = [int(nums[0]), splitstr(valves[3:])]
            
            
    return res

#https://onestepcode.com/graph-shortest-path-python/
def shortest_path(graph, node1, node2):
    """
    This function returns the shortest path from node 1 to node 2
    """
    path_list = [[node1]]
    path_index = 0
    
    previous_nodes = {node1}
    if node1 == node2:
        return path_list[0]
        
    while path_index < len(path_list):
        current_path = path_list[path_index]
        last_node = current_path[-1]
        next_nodes = graph[last_node][1]
        # Search goal node
        if node2 in next_nodes:
            current_path.append(node2)
            return current_path
        # Add new paths
        for next_node in next_nodes:
            if not next_node in previous_nodes:
                new_path = current_path[:]
                new_path.append(next_node)
                path_list.append(new_path)
                # To avoid backtracking
                previous_nodes.add(next_node)
        # Continue to next path in list
        path_index += 1
    # No path is found
    return []
        

def dfs(tunnels, minutes, valves_to_open):
    """
    Try all possible paths and find the maximum 
    """
    stack = [[['AA'], 0, {}]]
    pressure_max = 0
    
    while len(stack):
        path, steps, opened = stack.pop()
        current_valve = path[-1]
        
        if steps >= minutes or len(path) == len(valves_to_open) + 1: # plus 1 because of AA:
            pressure_released = 0

            for valve, step in opened.items():
                step = max(minutes - step, 0)
                pressure_released += tunnels[valve][0] * step

            pressure_max = max(pressure_max, pressure_released)
        else:
            for valve in valves_to_open:
                if valve not in opened.keys():
                    shortest = shortest_path(tunnels, current_valve, valve)
                    move_time = len(shortest) - 1
                    new_steps = steps + move_time + 1 # spend 1 minute on opening the valve
                    new_opened = opened.copy()
                    new_opened[valve] = new_steps

                    new_path = list(path)
                    new_path.append(valve)

                    stack.append([new_path, new_steps, new_opened])
                    
    return pressure_max

def get_diff(list1, list2):
    return list(set(list1) - set(list2))

def get_subsets(list1):
    subsets = []
    for i in range(1, len(list1)//2):
        comb = combinations(list1, i) 
        subsets+=list(comb)
        
    return subsets


def dfs2(tunnels, minutes, valves_to_open):
    paths = defaultdict(int)
    stack = [[['AA'], 0, {}]]
    
    while len(stack):
        path, steps, opened = stack.pop()
        current_valve = path[-1]
        pressure_released = 0
        
        for valve, step in opened.items():
            step = max(minutes - step, 0)
            pressure_released += tunnels[valve][0] * step
        
        key = tuple(sorted(path))
        paths[key] = max(paths[key], pressure_released)
        
        
        if steps >= minutes or len(path) == len(valves_to_open) + 1: # plus 1 because of AA:
            valves_not_open = [valve for valve in valves_to_open if valve not in path]
            
            for r in range(1, len(valves_not_open)):
                for comb in combinations(sorted(valves_not_open), r):
                    key = tuple(sorted(path + list(comb)))
                    paths[key] = max(paths[key], pressure_released)

            continue

            
        else:
            for valve in valves_to_open:
                if valve not in opened.keys():
                    shortest = shortest_path(tunnels, current_valve, valve)
                    move_time = len(shortest) - 1
                    new_steps = steps + move_time + 1 # spend 1 minute on opening the valve
                    new_opened = opened.copy()
                    new_opened[valve] = new_steps

                    new_path = list(path)
                    new_path.append(valve)

                    stack.append([new_path, new_steps, new_opened])
                    
    return paths

In [ ]:
tunnels = read_txt(PATH+'/day16.txt')
# To releast the most pressure, try to open all valves whose flow rate is not 0
valves_to_open = [valve for valve in tunnels if tunnels[valve][0] > 0]

################################################ PART 1 ################################################
ans1 = dfs(tunnels, 30, valves_to_open)
print(f"ans1: {ans1}") # time consumed around 30 seconds


################################################ PART 2 ################################################
# 1. use DFS to get all possible paths and their maximum released pressure
# 2. get all subsets of valves_to_open, and assign them to human and elephant
# 3. compute the total pressure of all combinations and find the maximum
# 5. quite time consuming, around 5 mins, but I did not have time to try better solution

all_pressure = dfs2(tunnels, 26, valves_to_open)
all_sub =  get_subsets(valves_to_open)      

ans2 = 0

for subset in all_sub:
    human_pressure = all_pressure[tuple(sorted(['AA'] + list(subset)))]
    elephant_pressure = all_pressure[tuple(sorted(['AA'] + get_diff(valves_to_open, list(subset))))]

    ans2 = max(ans2, human_pressure + elephant_pressure)

print(f"ans2: {ans2}")
